In [159]:
import re
import requests
from bs4 import BeautifulSoup
# url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}' #기본 주소
url = 'https://comic.naver.com/webtoon/weekday.nhn' #기본 주소
response = requests.get(url)
html = BeautifulSoup(response.text)

In [167]:
# 전체 타이틀과 고유번호를 저장할 딕셔너리
webtoon_info = {}

def get_title_info(input_title, function):
    input_title_list = [] # 입력받은 값만 저장해 줄 리스트
    
    img_tag = html.select('div.thumb img[title]')
    
    # 타이틀과 고유번호 얻음
    for tag in img_tag:
        title = tag['title']
        title_id_tag = tag['src']
        title_id = re.findall(r'.*?webtoon/(\d+).*?', title_id_tag)[0]
    
        webtoon_info[title] = title_id # 딕셔너리에 값을 저장
    
    # 키와 값으로 필요한 값만 리스트에 따로 저장
    for key, value in webtoon_info.items():
        if input_title in key:
            input_title_list.append((key, value))
            
    for index, item in enumerate(input_title_list):
        print(f'{index+1}. {item[0]}')
    print('0. 이전\n')
    
    case = input('> 웹툰 선택: ')
    
    if (case == '0'): # 이전으로 갈 경우 아무동작 없이 종료
        return
    
    # 입력 값이 올바를 경우 고유 번호에 맞는 URL에 들어가서 html받아오기
    search_id = input_title_list[int(case)-1][1]
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={search_id}'
    response = requests.get(url)
    search_html = BeautifulSoup(response.text)
        
    function(search_html) # 각 동작에 따라 다른함수를 실행하게 함    

# 작가, 설명 등 상세정보 출력
def get_info(html):
    details = html.select_one('div.detail')
    detail = details.select_one('h2')
    
    # 각각의 디테일 정보 얻음
    title = detail.contents[0].strip()
    name = detail.select_one('span.wrt_nm').get_text(strip=True)
    string = details.select_one('p').get_text('\n')
    gern = details.select_one('span.genre').get_text()
    
    print(f'# 정보 ({title})\n작가: {name}\n장르: {gern}\n{string}\n')

# 에피소드의 제목, 이름, 작성일 출력
def get_episode(html):
    titles = html.select('td.title')
    ratings = html.select('div.rating_type strong')
    dates = html.select('td.num')
    for i in range(len(titles)):
        print(f'{titles[i].get_text(strip=True)}')
        print(f' 평점: {ratings[i].get_text()}, 날짜: {dates[i].get_text()}')
    print()

In [ ]:
while True:
    print('## 웹툰 크롤러 ##\n1. 정보\n2. 에피소드 목록\n0. 나가기\n')
    case = input('> 입력: ')
    
    if (case == '1'):
        title = input('> 웹툰명을 입력하세요: ')
        get_title_info(title, get_info)
    
    elif (case == '2'):
        title = input('> 웹툰명을 입력하세요: ')
        get_title_info(title, get_episode)
    
    elif (case == '0'):
        break
        
    else:
        print('입력이 틀렸습니다. 다시 입력하세요.')

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기

> 입력: 1
> 웹툰명을 입력하세요: 유
1. 유일무이 로맨스
2. 윌유메리미
3. 유미의 세포들
4. 공유몽
5. 유령극단
0. 이전

> 웹툰 선택: 1
# 정보 (유일무이 로맨스)
작가: 두부
장르: 스토리, 일상, 드라마, 로맨스
평범한 취준생 공유일의 일상에 
인기절정 탑배우 탁무이와 아이돌 류민이 침범해오기 시작했다!

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기

